# eEDB-011-2024-3

## Atividade 4 – Ingestão e ETL com linguagem de programação (Python + Spark + SQL)

* Utilizar linguagem de programação Python para ingestão e tratamento de dados. Para processo de transformação
deve ser realizado via SQL com uma das seguintes ferramentas:
  * DBT (indicada)
  *  SQL Mash – alternativa
  *  Coalesce - Alternativa

*  Base final deve ser um banco de dados relacional já utilizado nos exercícios anteriores.
* O processamento SQL deve ser utilizado via ferramenta mais uma engine de SQL que pode ser o próprio Banco de
Dados bem como um banco de dados em memória como Duckdb ou SLQLite, ou qualquer outra alternativa.
*  Gerar uma tabela final com os dados tratados e unidos.
  * O tratamento de dados deve ser realizado através da ferramenta escolhida para SQL
*  Adicionar as seguintes camadas de processamento, dentro do próprio banco de dados ou em disco local. A
Camada Delivery deve obrigatoriamente ter estar também no formato de uma tabela final dentro do banco de
dados relacional:
  *  RAW – formato dos dados livre
  *  Trusted – formato de dados em Parquet ou ORC or AVRO (indicado Parquet)
  *  Delivery– formato de dados em Parquet ou ORC or AVRO (indicado Parquet)

- **Grupo 02**:
    - Aline Bini
    - Ana Lívia Franco
    - Ana Priss
    - João Squinelato
    - Marcelo Pena
    - Thais Siqueira

- [Github](https://github.com/Squinelato/eEDB-011-2024-3 "eEDB-011-2024-3")

```Ingestão De Dados | Agosto 2024```

## To Do

- raw
- trusted
- delivery

In [1]:
%pip install pyspark
%pip install unidecode
%pip install findspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lpad, col, lpad, concat, sha1, regexp_replace, udf, lower, lit, when
from pyspark.sql.types import StringType, FloatType, IntegerType, StructType, StructField, ArrayType
from unidecode import unidecode

import findspark
import os

In [3]:
findspark.init()

In [4]:
spark = SparkSession.builder \
    .master('local') \
    .appName('Basic ETL') \
    .config('spark.executor.timeout', "1200s") \
    .config('spark.sql.broadcastTimeout', '1200s') \
    .config('spark.rpc.askTimeout', '600s') \
    .config('spark.executor.heartbeatInterval', '120s') \
    .config('spark.network.timeout', '1200s') \
    .getOrCreate()

In [5]:
spark

---
## **Raw**

### **Banks file**

In [67]:
# banks_csv_path = '../Fonte de Dados/Bancos/EnquadramentoInicia_v2.tsv'
banks_csv_path = '/content/drive/MyDrive/eEDB-011-2024-3/Fonte de Dados/Bancos/EnquadramentoInicia_v2.tsv'

# Leitura do arquivo CSV e criação da view temporária
bank_df = spark.read.csv(banks_csv_path, sep='\t', encoding='utf8', header=True)
bank_df.createOrReplaceTempView("bank_temp")
spark.sql("SELECT * FROM bank_temp LIMIT 100").show(truncate=False)

+--------+--------+---------------------------------------------+
|Segmento|CNPJ    |Nome                                         |
+--------+--------+---------------------------------------------+
|S1      |0       |BANCO DO BRASIL - PRUDENCIAL                 |
|S1      |60746948|BRADESCO - PRUDENCIAL                        |
|S1      |30306294|BTG PACTUAL - PRUDENCIAL                     |
|S1      |360305  |CAIXA ECONOMICA FEDERAL - PRUDENCIAL         |
|S1      |60872504|ITAU - PRUDENCIAL                            |
|S1      |90400888|SANTANDER - PRUDENCIAL                       |
|S2      |92702067|BANRISUL - PRUDENCIAL                        |
|S2      |7237373 |BANCO DO NORDESTE DO BRASIL S.A. - PRUDENCIAL|
|S2      |33657248|BNDES - PRUDENCIAL                           |
|S2      |33479023|CITIBANK - PRUDENCIAL                        |
|S2      |33987793|CREDIT SUISSE - PRUDENCIAL                   |
|S2      |58160789|SAFRA - PRUDENCIAL                           |
|S2      |

Analisando o esquema dos dados

In [68]:
spark.sql("DESCRIBE bank_temp").show(truncate=False)

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|Segmento|string   |NULL   |
|CNPJ    |string   |NULL   |
|Nome    |string   |NULL   |
+--------+---------+-------+



Contando a quantidade de linhas

In [69]:
total_records = spark.sql("SELECT COUNT(*) AS total FROM bank_temp").collect()[0]['total']
print(f"Total de registros: {total_records}")

Total de registros: 1474


Salvando dados na camada _raw_ no formato parquet

In [70]:
# raw_bank_path = './raw/bank/'
# rwzd_bank.write.parquet(raw_bank_path, mode="append")

spark.sql("""
CREATE TABLE rwzd_bank
USING parquet
LOCATION './raw/bank/'
AS
SELECT *
FROM bank_temp
""")
spark.sql("DESCRIBE rwzd_bank").show(truncate=False)

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|Segmento|string   |NULL   |
|CNPJ    |string   |NULL   |
|Nome    |string   |NULL   |
+--------+---------+-------+



### **Employees file**

Localizando todos os arquivos contendo dados de empregados

In [71]:
# employee_dir = '../Fonte de Dados/Empregados/'
employee_dir = '/content/drive/MyDrive/eEDB-011-2024-3/Fonte de Dados/Empregados'
employee_files = os.listdir(employee_dir)
employee_paths = list(map(lambda file: os.path.join(employee_dir, file), employee_files))[1]
employee_paths

'/content/drive/MyDrive/eEDB-011-2024-3/Fonte de Dados/Empregados/glassdoor_consolidado_join_match_v2.csv'

Lendo todos os arquivos de empregados como um único conjunto de dados

In [72]:
employee_df = spark.read.csv(employee_paths, sep='|', encoding='utf8', header=True)
employee_df.createOrReplaceTempView("employee_temp")
spark.sql("SELECT * FROM employee_temp LIMIT 100").show()

+--------------------+-------------+-------------+--------------+--------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+-----+-----------------+----------------------+-----------------+--------------+------------------------+-------------------------+---------------------------------+----------------------------------+--------+--------------------+-------------+
|       employer_name|reviews_count|culture_count|salaries_count|benefits_count|    employer-website|employer-headquarters|employer-founded|   employer-industry|    employer-revenue|                 url|Geral|Cultura e valores|Diversidade e inclusão|Qualidade de vida|Alta liderança|Remuneração e benefícios|Oportunidades de carreira|Recomendam para outras pessoas(%)|Perspectiva positiva da empresa(%)|Segmento|                Nome|match_percent|
+--------------------+-------------+-------------+--------------+--------------+--------------------+---

In [73]:
total_records_employee = spark.sql("SELECT COUNT(*) AS total FROM employee_temp").collect()[0]['total']
print(f"Total de registros: {total_records_employee}")

Total de registros: 34


Removendo duplicatas com base no nome e segmento do banco

In [74]:
# Remover duplicatas com base nas colunas 'Nome' e 'Segmento'
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW employee_temp_dedup AS
SELECT *
FROM (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY Nome, Segmento ORDER BY Nome) AS row_num
    FROM employee_temp
) tmp
WHERE row_num = 1
""")
spark.sql("SELECT * FROM employee_temp_dedup").show(truncate=False)

+-------------------------------+-------------+-------------+--------------+--------------+-------------------------------+---------------------------+----------------+-----------------------------------------------------------------------------------+---------------------------+----------------------------------------------------------------------------------------------------------------+-----+-----------------+----------------------+-----------------+--------------+------------------------+-------------------------+---------------------------------+----------------------------------+--------+--------------------------------------------------+-------------+-------+
|employer_name                  |reviews_count|culture_count|salaries_count|benefits_count|employer-website               |employer-headquarters      |employer-founded|employer-industry                                                                  |employer-revenue           |url                                         

In [75]:
total_records_employee_dedup = spark.sql("SELECT COUNT(*) AS total FROM employee_temp_dedup").collect()[0]['total']
print(f"Registros subtraídos: {total_records_employee - total_records_employee_dedup}")

Registros subtraídos: 2


Analisando o esquema dos dados

In [76]:
spark.sql("DESCRIBE employee_temp_dedup").show(truncate=False)

+----------------------------------+---------+-------+
|col_name                          |data_type|comment|
+----------------------------------+---------+-------+
|employer_name                     |string   |NULL   |
|reviews_count                     |string   |NULL   |
|culture_count                     |string   |NULL   |
|salaries_count                    |string   |NULL   |
|benefits_count                    |string   |NULL   |
|employer-website                  |string   |NULL   |
|employer-headquarters             |string   |NULL   |
|employer-founded                  |string   |NULL   |
|employer-industry                 |string   |NULL   |
|employer-revenue                  |string   |NULL   |
|url                               |string   |NULL   |
|Geral                             |string   |NULL   |
|Cultura e valores                 |string   |NULL   |
|Diversidade e inclusão            |string   |NULL   |
|Qualidade de vida                 |string   |NULL   |
|Alta lide

Salvando dados na camada _raw_ no formato parquet

In [77]:
# raw_employee_path = './raw/employee/'
# rwzd_employee.write.parquet(raw_employee_path, mode="append")

spark.sql("""
CREATE TABLE rwzd_employee
USING parquet
LOCATION './raw/employee/'
AS
SELECT *
FROM employee_temp_dedup
""")
spark.sql("DESCRIBE rwzd_employee").show(truncate=False)

+----------------------------------+---------+-------+
|col_name                          |data_type|comment|
+----------------------------------+---------+-------+
|employer_name                     |string   |NULL   |
|reviews_count                     |string   |NULL   |
|culture_count                     |string   |NULL   |
|salaries_count                    |string   |NULL   |
|benefits_count                    |string   |NULL   |
|employer-website                  |string   |NULL   |
|employer-headquarters             |string   |NULL   |
|employer-founded                  |string   |NULL   |
|employer-industry                 |string   |NULL   |
|employer-revenue                  |string   |NULL   |
|url                               |string   |NULL   |
|Geral                             |string   |NULL   |
|Cultura e valores                 |string   |NULL   |
|Diversidade e inclusão            |string   |NULL   |
|Qualidade de vida                 |string   |NULL   |
|Alta lide

### **Claims file**

Localizando todos os arquivos contendo dados de reclamações

In [8]:
# claim_dir = '../Fonte de Dados/Reclamações/'
claim_dir = '/content/drive/MyDrive/eEDB-011-2024-3/Fonte de Dados/Reclamações'
claim_files = os.listdir(claim_dir)
claim_paths = list(map(lambda file: os.path.join(claim_dir, file), claim_files))
claim_paths

['/content/drive/MyDrive/eEDB-011-2024-3/Fonte de Dados/Reclamações/2021_tri_02.csv',
 '/content/drive/MyDrive/eEDB-011-2024-3/Fonte de Dados/Reclamações/2021_tri_03.csv',
 '/content/drive/MyDrive/eEDB-011-2024-3/Fonte de Dados/Reclamações/2021_tri_01.csv',
 '/content/drive/MyDrive/eEDB-011-2024-3/Fonte de Dados/Reclamações/2022_tri_01.csv',
 '/content/drive/MyDrive/eEDB-011-2024-3/Fonte de Dados/Reclamações/2022_tri_02_nao_ha_dados.csv',
 '/content/drive/MyDrive/eEDB-011-2024-3/Fonte de Dados/Reclamações/2022_tri_03.csv',
 '/content/drive/MyDrive/eEDB-011-2024-3/Fonte de Dados/Reclamações/2021_tri_04.csv',
 '/content/drive/MyDrive/eEDB-011-2024-3/Fonte de Dados/Reclamações/2022_tri_04.csv']

In [22]:
claim_df = spark.read.csv(claim_paths, sep=';', encoding='latin1', header=True)
claim_df.createOrReplaceTempView("claim_temp")
spark.sql("SELECT * FROM claim_temp LIMIT 100").show(truncate=False)

+----+---------+------------------------------------------------------+----------------+--------+----------------------------------------------------------------+------+-----------------------------------------------+--------------------------------------------+---------------------------------------+-------------------------------+----------------------------------------+----------------------------+----------------------------+----+
|Ano |Trimestre|Categoria                                             |Tipo            |CNPJ IF |Instituição financeira                                          |Índice|Quantidade de reclamações reguladas procedentes|Quantidade de reclamações reguladas - outras|Quantidade de reclamações não reguladas|Quantidade total de reclamações|Quantidade total de clientes  CCS e SCR|Quantidade de clientes  CCS|Quantidade de clientes  SCR|_c14|
+----+---------+------------------------------------------------------+----------------+--------+-------------------------

Analisando o esquema dos dados

In [23]:
spark.sql("DESCRIBE claim_temp").show(truncate=False)

+-----------------------------------------------+---------+-------+
|col_name                                       |data_type|comment|
+-----------------------------------------------+---------+-------+
|Ano                                            |string   |NULL   |
|Trimestre                                      |string   |NULL   |
|Categoria                                      |string   |NULL   |
|Tipo                                           |string   |NULL   |
|CNPJ IF                                        |string   |NULL   |
|Instituição financeira                         |string   |NULL   |
|Índice                                         |string   |NULL   |
|Quantidade de reclamações reguladas procedentes|string   |NULL   |
|Quantidade de reclamações reguladas - outras   |string   |NULL   |
|Quantidade de reclamações não reguladas        |string   |NULL   |
|Quantidade total de reclamações                |string   |NULL   |
|Quantidade total de clientes  CCS e SCR       

Removendo coluna desnecessária

In [46]:
# Remove _c14 -> comando EXCEPT e DROP COLUMN nao funciona
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW claim_temp_filtered AS
SELECT "Ano", "Trimestre", "Categoria", "Tipo", "CNPJ IF", "Instituição financeira",
"Índice", "Quantidade de reclamações reguladas procedentes", "Quantidade de reclamações reguladas - outras",
"Quantidade de reclamações não reguladas", "Quantidade total de reclamações", "Quantidade total de clientes  CCS e SCR",
"Quantidade de clientes  CCS", "Quantidade de clientes  SCR"
FROM claim_temp
""")
spark.sql("DESCRIBE claim_temp_filtered").show(truncate=False)

+-----------------------------------------------+---------+-------+
|col_name                                       |data_type|comment|
+-----------------------------------------------+---------+-------+
|Ano                                            |string   |NULL   |
|Trimestre                                      |string   |NULL   |
|Categoria                                      |string   |NULL   |
|Tipo                                           |string   |NULL   |
|CNPJ IF                                        |string   |NULL   |
|Instituição financeira                         |string   |NULL   |
|Índice                                         |string   |NULL   |
|Quantidade de reclamações reguladas procedentes|string   |NULL   |
|Quantidade de reclamações reguladas - outras   |string   |NULL   |
|Quantidade de reclamações não reguladas        |string   |NULL   |
|Quantidade total de reclamações                |string   |NULL   |
|Quantidade total de clientes  CCS e SCR       

Contando a quantidade de linhas

In [48]:
total_records_claim= spark.sql("SELECT COUNT(*) AS total FROM claim_temp_filtered").collect()[0]['total']
print(f"Registros claim: {total_records_claim}")

Registros claim: 918


Salvando dados na camada _raw_ no formato parquet

In [62]:
# raw_claim_path = './raw/claim/'
# rwzd_claim.write.parquet(raw_claim_path, mode="append")

spark.sql("""
CREATE TABLE rwzd_claim
USING parquet
LOCATION './raw/claim/'
AS
SELECT *
FROM claim_temp_filtered
""")
spark.sql("DESCRIBE rwzd_claim").show(truncate=False)

+-----------------------------------------------+---------+-------+
|col_name                                       |data_type|comment|
+-----------------------------------------------+---------+-------+
|Ano                                            |string   |NULL   |
|Trimestre                                      |string   |NULL   |
|Categoria                                      |string   |NULL   |
|Tipo                                           |string   |NULL   |
|CNPJ IF                                        |string   |NULL   |
|Instituição financeira                         |string   |NULL   |
|Índice                                         |string   |NULL   |
|Quantidade de reclamações reguladas procedentes|string   |NULL   |
|Quantidade de reclamações reguladas - outras   |string   |NULL   |
|Quantidade de reclamações não reguladas        |string   |NULL   |
|Quantidade total de reclamações                |string   |NULL   |
|Quantidade total de clientes  CCS e SCR       

---
## **Trusted**

### **Banks**

In [ ]:
# rwzd_bank_df = spark.read.parquet('./raw/bank')
# rwzd_bank_df.createOrReplaceTempView("rwzd_bank")

Aplicando transformações para melhorar a qualidade dos dados:

1 - Renomeando colunas do dataframe para ingles e no formato snake_case

2 - Para que os dados da coluna _cnpj_ estivessem de acordo com seu padrão, os valores incompletos receberam numerais zeros à esquerda até completar 8 dígitos

In [98]:
spark.sql("""
CREATE TABLE trzd_banks
USING parquet
LOCATION './trusted/banks/'
AS
SELECT Segmento AS segment, Nome AS financial_institution_name,
      CASE
           WHEN cnpj = '' THEN NULL
           ELSE LPAD(cnpj, 8, '0')
       END AS cnpj
FROM rwzd_bank
""")
spark.sql("DESCRIBE trzd_banks").show(truncate=False)

+--------------------------+---------+-------+
|col_name                  |data_type|comment|
+--------------------------+---------+-------+
|segment                   |string   |NULL   |
|financial_institution_name|string   |NULL   |
|cnpj                      |string   |NULL   |
+--------------------------+---------+-------+



In [99]:
spark.sql("SELECT * FROM trzd_banks LIMIT 100").show(truncate=False)

+-------+---------------------------------------------+--------+
|segment|financial_institution_name                   |cnpj    |
+-------+---------------------------------------------+--------+
|S1     |BANCO DO BRASIL - PRUDENCIAL                 |00000000|
|S1     |BRADESCO - PRUDENCIAL                        |60746948|
|S1     |BTG PACTUAL - PRUDENCIAL                     |30306294|
|S1     |CAIXA ECONOMICA FEDERAL - PRUDENCIAL         |00360305|
|S1     |ITAU - PRUDENCIAL                            |60872504|
|S1     |SANTANDER - PRUDENCIAL                       |90400888|
|S2     |BANRISUL - PRUDENCIAL                        |92702067|
|S2     |BANCO DO NORDESTE DO BRASIL S.A. - PRUDENCIAL|07237373|
|S2     |BNDES - PRUDENCIAL                           |33657248|
|S2     |CITIBANK - PRUDENCIAL                        |33479023|
|S2     |CREDIT SUISSE - PRUDENCIAL                   |33987793|
|S2     |SAFRA - PRUDENCIAL                           |58160789|
|S2     |VOTORANTIM - PRU

### **Employees**

In [ ]:
# rwzd_employee_df = spark.read.parquet('./raw/employee')
# rwzd_employee_df.createOrReplaceTempView("rwzd_employee")

---
## **Delivery**